In [4]:
import requests
import bs4 as bs
import lxml

httpaddr = "https://www.lanacion.com.ar/"

requests_ret = requests.get(httpaddr)
print(requests_ret, end = ' || ')
soup = bs.BeautifulSoup(requests_ret.text, 'lxml')
# Notas:
# Everything semmantically a header, is tagged with <h2>. To all scrapping practical
# purposes, all headers except h2 are deemded irrelevant
articles = soup.find_all("article"); print("articles :", len(articles), end = ' / ')
sections = soup.find_all("section"); print("sections :", len(sections), end = ' / ')
headers = soup.find_all("header"); print("headers :", len(headers), end = ' / ')
h1s = soup.find_all("h1"); print("h1s :", len(h1s), end = ' / ')
h2s = soup.find_all("h2"); print("h2s :", len(h2s), end = ' / ')
h3s = soup.find_all("h3"); print("h3s :", len(h3s), end = ' / ')
h4s = soup.find_all("h4"); print("h4s :", len(h4s), end = ' / ')
h5s = soup.find_all("h5"); print("h5s :", len(h5s), end = ' / ')
h6s = soup.find_all("h6"); print("h6s :", len(h6s))
# only <h2>s hold relevance
# the only <h1> holds an SVG graphic

for discard_tag in ("script", "style"):
    for t in soup.find_all(discard_tag): t.extract()

<Response [200]> || articles : 110 / sections : 275 / headers : 1 / h1s : 1 / h2s : 107 / h3s : 21 / h4s : 2 / h5s : 0 / h6s : 0


In [5]:
# DUMPS A REPORT ON THE NESTING OF <article>s IN <section>s:
# TODO: This can be generalized as a routine that, for any two lists of nodes, gives a report on nesting

# Check for section nesting
# There's an almost strict hierarchical nesting in which articles are always inside some section
# The double belonging of to sections 2 and 3 are due to the fact that section 2 is nested inside section 3 (July 24, 2022)
print("+++ <section> NESTING CHECK :: ")
for (i, j) in [(i, j) for (i, section_i) in enumerate(sections) for (j, section_j) in enumerate(sections) if section_j in section_i.descendants]:
    print(("\t<section>[%" + str(len(str(len(sections)))) + "d] is nested in <section>[%" + str(len(str(len(sections)))) + "d]") % (j, i))

print("+++ <article> NESTING CHECK :: ")
for (i, j) in [(i, j) for (i, article_i) in enumerate(articles) for (j, article_j) in enumerate(articles) if article_j in article_i.descendants]:
    print(("\t<article>[%" + str(len(str(len(articles)))) + "d] is nested in <article>[%" + str(len(str(len(articles)))) + "d]") % (j, i))

# dangling <article>s
# these, the <article>s not-in-any-section, are the articles depicted as "LO MAS VISTO POR LOS SUSCRIPTORES" at the tail of the page
articles_not_in_sections = articles[:]              # copy articles ...
articles_in_sections_set = set()
for section in sections:
    for article in articles:
        if article in section.descendants:
            articles_in_sections_set = articles_in_sections_set | {article} # need to use this notation. set() notation consumes an interable and therefore iterates over article's members
for article in articles_in_sections_set:
    while article in articles_not_in_sections:      # while instead of if, just in case of repetition
        articles_not_in_sections.remove(article)    # ... then substract every article already under a <section>
print("+++ NUMBER OF <article>s NOT UNDER ANY <section> :", len(articles_not_in_sections))

# Report which <article> is nested in which <section>s
maxlen = max( (len('section[' + 'x' * len(str(len(sections))) + ']:') for section in sections) )
for i, section in enumerate(sections):
    line = (('section[%' + str(len(str(len(sections)))) + 'd]:' + ' ' * 100) % (i,))[:maxlen] + " ||"
    for article in articles: line += "X" if article in section.descendants else '.'; 
    line += "|"
    # get nearest <header> previous sibling -- filter only tags
    cursor = section
    while cursor and cursor.name != 'header':
        cursor = cursor.previous_sibling
        try:
            if cursor.name == 'header':
                line += ' ' + cursor.text.strip()
        except:
            pass

    # if prev_siblings_list:
    #     print("!!!")
    #     print(len(prev_siblings_list))
    #     while prev_siblings_list[0]:    # can do this if previous filtering succeded...
    #         if prev_siblings_list[0].name == 'header':
    #             line += ' ' + prev_siblings_list[0].text.strip()
    #             break
    #         prev_siblings_list = prev_siblings_list[1:]
    print(line)

+++ <section> NESTING CHECK :: 
	<section>[  5] is nested in <section>[  4]
	<section>[  6] is nested in <section>[  4]
	<section>[  7] is nested in <section>[  4]
	<section>[  8] is nested in <section>[  4]
	<section>[  9] is nested in <section>[  4]
	<section>[ 10] is nested in <section>[  4]
	<section>[ 12] is nested in <section>[ 11]
	<section>[ 13] is nested in <section>[ 11]
	<section>[ 14] is nested in <section>[ 11]
	<section>[ 15] is nested in <section>[ 11]
	<section>[ 16] is nested in <section>[ 11]
	<section>[ 17] is nested in <section>[ 11]
	<section>[ 18] is nested in <section>[ 11]
	<section>[ 19] is nested in <section>[ 11]
	<section>[ 20] is nested in <section>[ 11]
	<section>[ 21] is nested in <section>[ 11]
	<section>[ 22] is nested in <section>[ 11]
	<section>[ 23] is nested in <section>[ 11]
	<section>[ 24] is nested in <section>[ 11]
	<section>[ 26] is nested in <section>[ 25]
	<section>[ 27] is nested in <section>[ 25]
	<section>[ 28] is nested in <section>[ 25]


In [6]:
print(h1s)

[<h1 class="com-title --xl"><a class="com-link" href="/politica/el-gobierno-posterga-la-posibilidad-de-un-bono-o-un-aumento-fijo-para-los-trabajadores-asalariados-nid18102022/" title="Lo exige el kirchnerismo. El Gobierno posterga la posibilidad de un bono o un aumento fijo para los trabajadores asalariados"><span class="com-lead">Lo exige el kirchnerismo. </span>El Gobierno posterga la posibilidad de un bono o un aumento fijo para los trabajadores asalariados</a></h1>]
